### Our Work
我们首先是针对苹果，单独的图片进行一系列统一的操作，然后我们再进行识别；
操作包括：
降低亮度
降低太阳亮度的同时增加苹果的红色饱和度



### 美图秀秀操作逻辑
我们做的是降低亮度的操作，究竟有没有效果，我们还不知道！

In [ ]:
#这段代码最关键了，对所有图片都进习惯一样的曝光操作
import time
import numpy as np
import cv2
import os

def create_directory(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)
 
def lighting(img, light):
    assert -100 <= light <= 100
    max_v = 4
    bright = (light/100.0)/max_v
    mid = 1.0+max_v*bright
    print('bright: ', bright, 'mid: ', mid)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY).astype(np.float32)/255.0
    thresh = gray*gray
    t = np.mean(thresh)
    mask = np.where(thresh > t, 255, 0).astype(np.float32)
    brightrate = np.zeros_like(mask).astype(np.float32)
    h, w = img.shape[:2]
    # 遍历每个像素点
    for i in range(h):
        for j in range(w):
            if mask[i, j] == 255.0:
                mask[i, j] = mid
                brightrate[i, j] = bright
            else:
                mask[i, j] = (mid-1.0)/t*thresh[i, j]+1.0
                brightrate[i, j] = (1.0/t*thresh[i, j])*bright
    img = img/255.0
    img = np.power(img, 1.0/mask[:, :, np.newaxis])*(1.0/(1.0-brightrate[:, :, np.newaxis]))
    img = np.clip(img, 0, 1.0)*255.0
    return img.astype(np.uint8)
 
 
def lighting_fast(img, light):
    assert -100 <= light <= 100
    max_v = 4
    bright = (light/100.0)/max_v
    mid = 1.0+max_v*bright
    print('bright: ', bright, 'mid: ', mid)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY).astype(np.float32)/255.0
    thresh = gray*gray
    t = np.mean(thresh)
    # 使用numpy来计算可以加速，速度远快于上面的遍历
    mask = np.where(thresh > t, 255, 0).astype(np.float32)
    brightrate = np.where(mask == 255.0, bright, (1.0/t*thresh)*bright)
    mask = np.where(mask == 255.0, mid, (mid-1.0)/t*thresh+1.0)
    img = img/255.0
    img = np.power(img, 1.0/mask[:, :, np.newaxis])*(1.0/(1.0-brightrate[:, :, np.newaxis]))
    img = np.clip(img, 0, 1.0)*255.0
    return img.astype(np.uint8)
 
 

if __name__ == '__main__':
    # 添加循环处理多张图片的代码
    input_directory = 'Attachment_1'
    output_directory = 'Attachment_2'
    light = -75

    for image_index in range(1, 201):
        input_img = cv2.imread(f'{input_directory}/{image_index}.jpg')
        start_time = time.time()
        res = lighting_fast(input_img, light)
        print(f'处理第 {image_index} 张图片，耗时: {time.time() - start_time:.3f} s')
        create_directory(output_directory)
        cv2.imwrite(f'{output_directory}/{image_index}_processed.jpg', res)

### 分水岭

In [ ]:
#这段代码实现的是使用分水岭后的结果
import cv2
import numpy as np
import matplotlib.pyplot as plt

# 读取图片
def ReadImg(image_path):
    img = cv2.imread(image_path, 1)
    return img

# 高斯滤波
def GausBlur(src):
    dst = cv2.GaussianBlur(src, (5, 5), 1.5)
    return dst

# 开运算（替代 open_mor）
def open_mor(img):
    kernel = np.ones((5, 5), np.uint8)
    result = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel)
    return result

# 红色对象检测
def detect_red_objects(src):
    # 设定红色阈值范围
    lower_red = np.array([0, 0, 100])
    upper_red = np.array([100, 100, 255])

    # 根据阈值构建掩模
    mask = cv2.inRange(src, lower_red, upper_red)

    # 使用掩模提取红色对象
    red_objects = cv2.bitwise_and(src, src, mask=mask)

    return red_objects

# 轮廓拟合
def draw_shape(open_img, src, markers):
    contours, hierarchy = cv2.findContours(open_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    apple_count = 0
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area > 50:  # 假设红色对象的最小面积为100
            # 获取圆心和半径
            (x, y), radius = cv2.minEnclosingCircle(cnt)
            center = (int(x), int(y))
            radius = int(radius)

            # 绘制圆形
            cv2.circle(src, center, radius, (0, 0, 255), 2)

            apple_count += 1

    return src, apple_count

# 处理所有图像
for i in range(1, 201):  # 假设图像编号从1到200
    image_path = f'Attachment_2/{i}_processed.jpg'
    src = ReadImg(image_path)
    gaus_img = GausBlur(src)
    red_objects_img = detect_red_objects(gaus_img)
    gray_img = cv2.cvtColor(red_objects_img, cv2.COLOR_BGR2GRAY)
    _, thres_img = cv2.threshold(gray_img, 1, 255, cv2.THRESH_BINARY)
    open_img = open_mor(thres_img)

    # 分水岭
    sure_bg = cv2.dilate(open_img, None, iterations=3)
    dist_transform = cv2.distanceTransform(open_img, cv2.DIST_L2, 5)
    _, sure_fg = cv2.threshold(dist_transform, 0.7 * dist_transform.max(), 255, 0)
    sure_fg = np.uint8(sure_fg)
    unknown = cv2.subtract(sure_bg, sure_fg)
    _, markers = cv2.connectedComponents(sure_fg)
    markers = markers + 1
    markers[unknown == 255] = 0
    markers = cv2.watershed(src, markers)

    # Use watershed result for counting
    result_img, apple_count = draw_shape(open_img, src, markers)

    # 显示图像
    plt.imshow(cv2.cvtColor(result_img, cv2.COLOR_BGR2RGB))
    plt.title(f'Processed Image {i}, Red Objects Count: {apple_count}')
    plt.show()

### 圈出苹果+显示坐标

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

# 读取图片
def ReadImg(image_path):
    img = cv2.imread(image_path, 1)
    return img

# 高斯滤波
def GausBlur(src):
    dst = cv2.GaussianBlur(src, (5, 5), 1.5)
    return dst

# 开运算（替代 open_mor）
def open_mor(img):
    kernel = np.ones((5, 5), np.uint8)
    result = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel)
    return result

# 红色对象检测
def detect_red_objects(src):
    # 设定红色阈值范围
    lower_red = np.array([0, 0, 100])
    upper_red = np.array([100, 100, 255])

    # 根据阈值构建掩模
    mask = cv2.inRange(src, lower_red, upper_red)

    # 使用掩模提取红色对象
    red_objects = cv2.bitwise_and(src, src, mask=mask)

    return red_objects

# 轮廓拟合
def draw_shape(open_img, src):
    contours, hierarchy = cv2.findContours(open_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    apple_count = 0
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area > 50:  # 假设红色对象的最小面积为100
            # 获取圆心和半径
            (x, y), radius = cv2.minEnclosingCircle(cnt)
            center = (int(x), int(y))
            radius = int(radius)

            # 描绘绿色的边框线
            cv2.drawContours(src, [cnt], 0, (0, 255, 0), 2)

            # 绘制圆形
            cv2.circle(src, center, radius, (0, 0, 255), 2)

            apple_count += 1

    return src, apple_count

# Watershed 分割
def watershed_segmentation(open_img, src):
    # 使用距离变换和阈值分割进行分水岭算法
    dist_transform = cv2.distanceTransform(open_img, cv2.DIST_L2, 5)
    _, sure_fg = cv2.threshold(dist_transform, 0.7 * dist_transform.max(), 255, 0)
    sure_fg = np.uint8(sure_fg)

    # 找到未知区域
    sure_bg = cv2.dilate(open_img, None, iterations=3)
    sure_bg = 255 - sure_bg

    # 标记未知区域
    _, markers = cv2.connectedComponents(sure_fg)
    markers = markers + 1
    markers[open_img == 0] = 0

    # 应用分水岭算法
    cv2.watershed(src, markers)

    return src

# 处理所有图像
for i in range(1, 201):  # 假设图像编号从1到200
    image_path = f'Attachment_2/{i}_processed.jpg'
    src = ReadImg(image_path)
    gaus_img = GausBlur(src)
    red_objects_img = detect_red_objects(gaus_img)
    gray_img = cv2.cvtColor(red_objects_img, cv2.COLOR_BGR2GRAY)
    _, thres_img = cv2.threshold(gray_img, 1, 255, cv2.THRESH_BINARY)
    open_img = open_mor(thres_img)
    result_img, apple_count = draw_shape(open_img, src)

    # Watershed 分割
    result_img = watershed_segmentation(open_img, result_img)

    # 上下翻转图像，不影响Y轴
    flipped_img = np.flipud(result_img)

    # 处理所有图像
    output_folder = 'abcde'
    os.makedirs(output_folder, exist_ok=True)
    # 创建带有坐标轴的图像
    plt.imshow(cv2.cvtColor(flipped_img, cv2.COLOR_BGR2RGB))
    plt.gca().invert_yaxis()  # 反转Y轴
    plt.xticks([])  # 移除X轴坐标标签
    plt.yticks([])  # 移除Y轴坐标标签
    plt.title(f'Processed Image {i}, Red Objects Count: {apple_count}')

    # 将带有坐标轴的图像保存到文件
    output_path = os.path.join(output_folder, f'output_image_{i}.png')
    plt.savefig(output_path)
    plt.clf()  # 清除当前图像，以便下一次使用

    print(f'保存图像到: {output_path}')

    # 显示翻转后的图像
    plt.imshow(cv2.cvtColor(flipped_img, cv2.COLOR_BGR2RGB))
    plt.gca().invert_yaxis()  # 反转Y轴
    plt.xticks([])  # 移除X轴坐标标签
    plt.yticks([])  # 移除Y轴坐标标签
    plt.title(f'Processed Image {i}, Red Objects Count: {apple_count}')
    plt.show()

[ WARN:0@0.220] global loadsave.cpp:241 findDecoder imread_('Attachment_2/1_processed.jpg'): can't open/read file: check file path/integrity


error: OpenCV(4.10.0) /Users/xperience/GHA-Actions-OpenCV/_work/opencv-python/opencv-python/opencv/modules/imgproc/src/smooth.dispatch.cpp:617: error: (-215:Assertion failed) !_src.empty() in function 'GaussianBlur'


### 蒙特卡洛算法估计苹果质量和体积


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

# 读取图片
def ReadImg(image_path):
    img = cv2.imread(image_path, 1)
    return img

# 高斯滤波
def GausBlur(src):
    dst = cv2.GaussianBlur(src, (5, 5), 1.5)
    return dst

# 开运算（替代 open_mor）
def open_mor(img):
    kernel = np.ones((5, 5), np.uint8)
    result = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel)
    return result

# 红色对象检测
def detect_red_objects(src):
    # 设定红色阈值范围
    lower_red = np.array([0, 0, 100])
    upper_red = np.array([100, 100, 255])

    # 根据阈值构建掩模
    mask = cv2.inRange(src, lower_red, upper_red)

    # 使用掩模提取红色对象
    red_objects = cv2.bitwise_and(src, src, mask=mask)

    return red_objects

# 轮廓拟合
def process_and_save_image(open_img, src, image_path, i):
    contours, hierarchy = cv2.findContours(open_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    apple_count = 0
    with open('apple_coordinates.txt', 'a') as file:
        for cnt in contours:
            area = cv2.contourArea(cnt)
            if area > 50:  # 假设红色对象的最小面积为100
                # 获取圆心和半径
                (x, y), radius = cv2.minEnclosingCircle(cnt)
                center = (int(x), int(y))
                radius = int(radius)

                # 在文件中写入坐标
                file.write(f'({x}, {y})\n')
                apple_count += 1

    # 绘制图像
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area > 50:  # 假设红色对象的最小面积为100
            # 获取圆心和半径
            (x, y), radius = cv2.minEnclosingCircle(cnt)
            center = (int(x), int(y))
            radius = int(radius)

            # 绘制圆形
            cv2.circle(src, center, radius, (0, 0, 255), 2)



# 创建结果图像保存文件夹
output_folder = 'Processed_Images'
os.makedirs(output_folder, exist_ok=True)

# 处理所有图像
for i in range(1, 201):  # 假设图像编号从1到200
    image_path = f'Attachment_2/{i}_processed.jpg'
    src = ReadImg(image_path)
    gaus_img = GausBlur(src)
    red_objects_img = detect_red_objects(gaus_img)
    gray_img = cv2.cvtColor(red_objects_img, cv2.COLOR_BGR2GRAY)
    _, thres_img = cv2.threshold(gray_img, 1, 255, cv2.THRESH_BINARY)
    open_img = open_mor(thres_img)
    apple_count = process_and_save_image(open_img, src, image_path, i)

    # 保存结果图像到新的文件夹
    output_path = os.path.join(output_folder, f'Processed_Image_{i}.jpg')
    cv2.imwrite(output_path, src)

    print(f'Processed Image {i}, Red Objects Count: {apple_count}')

### 散点绘图

import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

# 读取自己的图片作为背景
custom_background_img = np.array(Image.open("/Users/zhangying/Processed_Image_2.jpg"))
# 创建一个函数来生成图像并添加随机点
def generate_image(base_image, num_points):
    img = np.copy(base_image)  # 创建基于基础图片的副本
    height, width = img.shape

    # 在图像上随机生成指定数量的点
    for _ in range(num_points):
        x, y = np.random.randint(0, width), np.random.randint(0, height)
        img[y, x] = 0  # 在随机位置添加黑色点

    return img

# 生成四个不同数量随机点的图片
num_points_list = [100, 500, 1500, 3000]
images = [generate_image(background_img, num_points) for num_points in num_points_list]

plt.figure(figsize=(8, 8))

for i in range(4):
    plt.subplot(2, 2, i + 1)
    
    # 获取随机点的坐标
    coords = np.argwhere(images[i] == 0)
    
    # 绘制散点图，设置颜色为绿色
    plt.scatter(coords[:, 1], coords[:, 0], c='green', s=5)  # y坐标在前，x坐标在后
    # 显示自定义背景图片
    plt.imshow(custom_background_img, cmap='gray', alpha=1)  # 设置alpha值透明度
    plt.title(f'{num_points_list[i]} Random Points')
    plt.axis('off')  # 关闭坐标轴

plt.tight_layout()
plt.show()

### 成熟度检测

In [ ]:
### 成熟度检测
import keras
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import numpy as np

#2类分类
num_classes = 2
nppath = "./"

#加载数据
x_train = np.load(nppath + "x_train.npy")
# x_test = np.load(nppath + "x_train.npy")
x_test = np.load(nppath + "x_test.npy")
y_train = np.load(nppath + "y_train.npy")
# y_test = np.load(nppath + "y_train.npy")
y_test = np.load(nppath + "y_test.npy")

x_train = x_train.astype('float32')/255
x_test = x_test.astype('float32')/255

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()

model.add(Conv2D(32, (3, 3), padding='same', input_shape=x_train.shape[1:]))
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())

model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.summary()

# 此优化器对本实验不适用
#opt = keras.optimizers.rmsprop(lr=0.001, decay=1e-6)

# train the model using RMSprop
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# hist = model.fit(x_train, y_train, epochs=100, shuffle=True)
# model.save("./cnnmodel.h5")
#
# # evaluate
# loss, accuracy = model.evaluate(x_test, y_test)
# print(loss, accuracy)

import matplotlib.pyplot as plt

def plot_loss(history):
    # 提取训练集和验证集的损失值
    train_loss = history.history['loss']
    val_loss = history.history['val_loss']

    train_accuracy = history.history['accuracy']
    val_accuracy = history.history['val_accuracy']

    # 设置图表标题和标签
    plt.title('Model Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')

    # 绘制训练集和验证集的损失曲线
    plt.plot(train_loss, label='Training Loss', color='blue')
    plt.plot(val_loss, label='Validation Loss', color='orange')

    # 添加图例
    plt.legend()

    # 显示图表
    plt.show()


def plot_acc(history):
    # 提取训练集和验证集的准确率
    train_accuracy = history.history['accuracy']
    val_accuracy = history.history['val_accuracy']

    # 设置图表标题和标签
    plt.title('Model Acc')
    plt.xlabel('Epoch')
    plt.ylabel('Acc')

    # 绘制训练集和验证集的损失曲线
    plt.plot(train_accuracy, label='Training Acc', color='red')
    plt.plot(val_accuracy, label='Validation Acc', color='yellow')

    # 添加图例
    plt.legend()

    # 显示图表
    plt.show()

# 训练模型并记录训练历史
history = model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=20, batch_size=16, verbose=0)

# 绘制损失曲线
plot_loss(history)
plot_acc(history)

### 训练模型，看Attachment 3 的苹果是否预测准确


In [ ]:
import torch
from torchvision import models, transforms
from torchvision.models import resnet50
from PIL import Image
import os
import re
import matplotlib.pyplot as plt

def extract_numbers_in_brackets(input_string):
    # 使用正则表达式匹配括号内的数字
    pattern = r'\((\d+)\)'
    matches = re.findall(pattern, input_string)

    # 返回匹配到的数字列表
    return [int(match) for match in matches][0]


# 加载预训练的ResNet-50模型
model = resnet50()
model_path = './checkpoint/fruit_reg.pth'
# model.load_state_dict(torch.load(model_path))
model = torch.load(model_path)
# print(net)
model.eval()  # 将模型设置为评估模式，这是因为在推理阶段不需要梯度

# 定义数据转换
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # ResNet-50的输入尺寸为224x224
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# 获取指定路径下的所有文件和文件夹
directory = '/Users/wuguobin/Documents/model/Attachment/Attachment 3/'
files = os.listdir(directory)

appleID = []
# 遍历文件和文件夹
for file in files:
    # 获取文件或文件夹的完整路径
    image_path = os.path.join(directory, file)

    # 加载图像并进行预测
    # image_path = '/Users/wuguobin/Documents/model/Attachment/Attachment 3/Fruit (75).jpg'
    image = Image.open(image_path)
    input_tensor = transform(image)
    input_batch = input_tensor.unsqueeze(0)  # 添加一个维度表示批次大小

    # 使用模型进行预测
    with torch.no_grad():
        output = model(input_batch)

    # 加载类标签文件
    # with open('imagenet_classes.txt', 'r') as f:
    #     classes = [line.strip() for line in f.readlines()]

    # 获取预测结果
    _, predicted_idx = torch.max(output, 1)
    # print(predicted_idx)
    # predicted_label = classes[predicted_idx.item()]
    # print(f'Predicted label: {predicted_label}')

    # 如果是苹果，记录苹果ID
    if predicted_idx==0:
        num = extract_numbers_in_brackets(image_path)
        print(num)
        appleID.append(num)

print(len(appleID))

# 绘制直方图
plt.hist(appleID, bins=50, color='skyblue', edgecolor='black')  # bins为柱子数量
plt.xlabel('ID numbers of all apple images')
plt.ylabel('freq')
plt.title('distribution histogram')
plt.show()

### 识别出青苹果和红苹果


In [ ]:
import cv2
import numpy as np
import glob
import os

# 创建文件夹以存储处理后的图像
output_folder = "output_images"
os.makedirs(output_folder, exist_ok=True)

def process_image(image_path, output_folder):
    image = cv2.imread(image_path)

    # 以下是图像处理的代码...

    # 保存中间和最终图像
    image_name = os.path.splitext(os.path.basename(image_path))[0]

    # 保存苹果图像
    cv2.imwrite(os.path.join(output_folder, f"{image_name}_apples.jpg"), apples)

    # 保存掩膜图像
    cv2.imwrite(os.path.join(output_folder, f"{image_name}_mask.jpg"), mask)

    # 保存高斯模糊图像
    cv2.imwrite(os.path.join(output_folder, f"{image_name}_GaussianBlur.jpg"), mask)

    # 保存腐蚀和膨胀图像
    cv2.imwrite(os.path.join(output_folder, f"{image_name}_erode_dilate.jpg"), mask)

    # 保存绘制轮廓后的图像
    cv2.imwrite(os.path.join(output_folder, f"{image_name}_draw_img.jpg"), draw_img)

    print(f"处理并保存完成：{image_name}")

# 读取图像
image_paths = glob.glob('Attachment_1/*.jpg')  # 修改为你的图像文件夹路径

for image_path in image_paths:
    process_image(image_path, output_folder)

### 苹果面积估计

In [ ]:
#这个是仅仅打印的，看看能不能用吧
import random
import cv2
import numpy as np
import glob

def is_point_inside_polygon(x, y, polygon):
    n = len(polygon)
    inside = False

    p1x, p1y = polygon[0][0]
    for i in range(n + 1):
        p2x, p2y = polygon[i % n][0]
        if y > min(p1y, p2y) and y <= max(p1y, p2y) and x <= max(p1x, p2x):
            if p1y != p2y:
                xinters = (y - p1y) * (p2x - p1x) / (p2y - p1y) + p1x
                if p1x == p2x or x <= xinters:
                    inside = not inside
        p1x, p1y = p2x, p2y

    return inside

def monte_carlo_pi(num, polygon, x1, y1, x2, y2):
    points_inside_circle = 0

    for _ in range(num):
        x = random.uniform(x1, x2)
        y = random.uniform(y1, y2)

        if is_point_inside_polygon(x, y, polygon):
            points_inside_circle += 1

    w = abs(x1 - x2)
    h = abs(y1 - y2)
    pi_estimate = points_inside_circle / num * w * h
    return pi_estimate

# 获取图像文件路径列表
image_folder_path = '/Users/zhangying/iCloud Drive (Archive) - 2/Desktop/Andrea/Academic 学术/建模资料/亚太地区A题解题/Attachment 1'  # 修改为你的图像文件夹路径
image_paths = glob.glob(image_folder_path + '/*.jpg')

for image_path in image_paths:
    image = cv2.imread(image_path)

    lower_green = np.array([40, 40, 40])
    upper_green = np.array([80, 255, 255])

    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    mask_green = cv2.inRange(hsv, lower_green, upper_green)

    kernel = np.ones((5, 5), np.uint8)
    mask_green = cv2.morphologyEx(mask_green, cv2.MORPH_CLOSE, kernel)
    mask_green = cv2.morphologyEx(mask_green, cv2.MORPH_OPEN, kernel)
    mask_apple = cv2.bitwise_not(mask_green)
    apples = cv2.bitwise_and(image, image, mask=mask_apple)

    hsv = cv2.cvtColor(apples, cv2.COLOR_BGR2HSV)

    lower_red1 = np.array([0, 100, 150])
    upper_red1 = np.array([10, 255, 255])
    lower_red2 = np.array([160, 100, 100])
    upper_red2 = np.array([180, 255, 255])

    mask1 = cv2.inRange(hsv, lower_red1, upper_red1)
    mask2 = cv2.inRange(hsv, lower_red2, upper_red2)
    mask = cv2.bitwise_or(mask1, mask2)

    mask = cv2.GaussianBlur(mask, (5, 5), 0)

    kernel = np.ones((5, 5), np.uint8)
    mask = cv2.erode(mask, kernel, iterations=2)
    mask = cv2.dilate(mask, kernel, iterations=2)

    contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

    all_area_estimate = 0
    for i in range(len(contours)):
        cnt = contours[i]
        x, y, w, h = cv2.boundingRect(cnt)
        area_estimate = monte_carlo_pi(1000, cnt, x, y, x+w, y+h)
        all_area_estimate += area_estimate

    image_name = image_path.split('/')[-1].split('.')[0]
    print(f"图像 {image_name} 的苹果总面积估算值为：{all_area_estimate}")

In [ ]:
#现在试试用Excel
import random
import cv2
import numpy as np
import glob
import pandas as pd

def is_point_inside_polygon(x, y, polygon):
    n = len(polygon)
    inside = False

    p1x, p1y = polygon[0][0]
    for i in range(n + 1):
        p2x, p2y = polygon[i % n][0]
        if y > min(p1y, p2y) and y <= max(p1y, p2y) and x <= max(p1x, p2x):
            if p1y != p2y:
                xinters = (y - p1y) * (p2x - p1x) / (p2y - p1y) + p1x
                if p1x == p2x or x <= xinters:
                    inside = not inside
        p1x, p1y = p2x, p2y

    return inside

def monte_carlo_pi(num, polygon, x1, y1, x2, y2):
    points_inside_circle = 0

    for _ in range(num):
        x = random.uniform(x1, x2)
        y = random.uniform(y1, y2)

        if is_point_inside_polygon(x, y, polygon):
            points_inside_circle += 1

    w = abs(x1 - x2)
    h = abs(y1 - y2)
    pi_estimate = points_inside_circle / num * w * h
    return pi_estimate

# 获取图像文件路径列表
image_folder_path = '/Users/zhangying/iCloud Drive (Archive) - 2/Desktop/Andrea/Academic 学术/建模资料/亚太地区A题解题/Attachment 1'  # 修改为你的图像文件夹路径
image_paths = glob.glob(image_folder_path + '/*.jpg')
# 创建一个空的列表，用于存储结果
result_list = []

for image_path in image_paths:
    image = cv2.imread(image_path)

    lower_green = np.array([40, 40, 40])
    upper_green = np.array([80, 255, 255])

    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    mask_green = cv2.inRange(hsv, lower_green, upper_green)

    kernel = np.ones((5, 5), np.uint8)
    mask_green = cv2.morphologyEx(mask_green, cv2.MORPH_CLOSE, kernel)
    mask_green = cv2.morphologyEx(mask_green, cv2.MORPH_OPEN, kernel)
    mask_apple = cv2.bitwise_not(mask_green)
    apples = cv2.bitwise_and(image, image, mask=mask_apple)

    hsv = cv2.cvtColor(apples, cv2.COLOR_BGR2HSV)

    lower_red1 = np.array([0, 100, 150])
    upper_red1 = np.array([10, 255, 255])
    lower_red2 = np.array([160, 100, 100])
    upper_red2 = np.array([180, 255, 255])

    mask1 = cv2.inRange(hsv, lower_red1, upper_red1)
    mask2 = cv2.inRange(hsv, lower_red2, upper_red2)
    mask = cv2.bitwise_or(mask1, mask2)

    mask = cv2.GaussianBlur(mask, (5, 5), 0)

    kernel = np.ones((5, 5), np.uint8)
    mask = cv2.erode(mask, kernel, iterations=2)
    mask = cv2.dilate(mask, kernel, iterations=2)

    contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

    all_area_estimate = 0
    for i in range(len(contours)):
        cnt = contours[i]
        x, y, w, h = cv2.boundingRect(cnt)
        area_estimate = monte_carlo_pi(1000, cnt, x, y, x+w, y+h)
        all_area_estimate += area_estimate

    image_name = image_path.split('/')[-1].split('.')[0]
    num_of_apples = len(contours)
 # 将结果添加到列表中
    result_list.append({'Image': image_name,
                        'Number of Apples': num_of_apples,
                        'Total Area Estimate': all_area_estimate})

# 将列表转换为DataFrame
result_df = pd.DataFrame(result_list)

# 将DataFrame写入EXCEL文件
output_csv_path = '蒙特卡洛.xlsx'  # 修改为你的输 出EXCEL文件路径
result_df.to_excel(output_csv_path, index=False)

print(f"结果已写入EXCEL文件：{output_csv_path}")